**Необходимые импорты**

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(2)
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from PIL import Image, ImageChops, ImageEnhance, ImageOps
import os
import itertools

**Инверсия**

In [2]:
def invert(image):
    return ImageOps.invert(image)

**Контраст**

In [3]:
def change_contrast(img, level):
    factor = (259 * (level + 255)) / (255 * (259 - level))
    def contrast(c):
        return 128 + factor * (c - 128)
    return img.point(contrast)

**Анализ уровня ошибок**

In [4]:
def convert_to_ela_image(path, quality):
    temp_filename = 'temp_file_name.jpg'
    ela_filename = 'temp_ela.png'
    
    image = Image.open(path).convert('RGB')
    #image = change_contrast(image, 100)
    #brightness_factor = 1.5
    #enhancer = ImageEnhance.Brightness(image)
    #img = enhancer.enhance(brightness_factor)
    #image = change_contrast(img, 100)
    image.save(temp_filename, 'JPEG', quality = quality)
    temp_image = Image.open(temp_filename)
    
    ela_image = ImageChops.difference(image, temp_image)
    
    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)
    
    return ela_image

**Подготовка датасета**

In [5]:
image_size = (128, 128)

In [6]:
def prepare_image(image_path):
    return np.array(convert_to_ela_image(image_path, 90).resize(image_size)).flatten() / 255.0

In [7]:
X = [] # Изображения после ELA
Y = [] # 0 - отфотошопленные, 1 - оригинальные

**Выбор n случайных изображений**

Оригинальные

In [8]:
import random
path = '/kaggle/input/casia-20-image-tampering-detection-dataset/CASIA2/Au'
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        if filename.endswith('jpg') or filename.endswith('png'):
            full_path = os.path.join(dirname, filename)
            X.append(prepare_image(full_path))
            Y.append(1)
            if len(Y) % 200 == 0:
                print(f'Processing {len(Y)} images')

random.shuffle(X)
X = X[:2600]
Y = Y[:2600]
print(len(X), len(Y))

Отредактированные

In [9]:
path = '/kaggle/input/casia-20-image-tampering-detection-dataset/CASIA2/Tp'
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        if filename.endswith('jpg') or filename.endswith('png'):
            full_path = os.path.join(dirname, filename)
            X.append(prepare_image(full_path))
            Y.append(0)
            if len(Y) % 200 == 0:
                print(f'Processing {len(Y)} images')

print(len(X), len(Y))

In [10]:
X = np.array(X)
Y = to_categorical(Y, 2)
X = X.reshape(-1, 128, 128, 3)

**Train/Test - 80/20**

In [11]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state=5)
X = X.reshape(-1,1,1,1)
print(len(X_train), len(Y_train))
print(len(X_val), len(Y_val))

**Построение модели**

In [12]:
def build_model():
    model = Sequential()
    model.add(Conv2D(input_shape=(128,128,3),filters=64,kernel_size=(5,5),padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(5,5),padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=(5,5), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(5,5), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(5,5), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(5,5), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(5,5), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(5,5), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(5,5), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(5,5), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(5,5), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(5,5), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(5,5), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Flatten())
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dense(units=2, activation="softmax"))
    return model

In [13]:
model = build_model()
model.summary()

In [14]:
epochs = 30
batch_size = 32
init_lr = 1e-4
optimizer = Adam(lr = init_lr, decay = init_lr/epochs)

In [15]:
model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [16]:
early_stopping = EarlyStopping(monitor = 'val_acc',
                              min_delta = 0,
                              patience = 2,
                              verbose = 0,
                              mode = 'auto')

In [17]:
hist = model.fit(X_train,
                 Y_train,
                 batch_size = batch_size,
                 epochs = epochs,
                validation_data = (X_val, Y_val),
                callbacks = [early_stopping])

In [18]:
model.save('VGG-16, 30ep, le1e-4, 2600 CASIA-2.h5')

**Метрики**

In [19]:
fig, ax = plt.subplots(2,1)
fig.set_size_inches(10, 5)
fig.set_dpi(100)
ax[0].plot(hist.history['loss'], color='b', label="Training loss")
ax[0].plot(hist.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(hist.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(hist.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

Матрица ошибок

In [20]:
Y_pred = model.predict(X_val)
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
Y_true = np.argmax(Y_val,axis = 1) 

In [21]:
sns.heatmap(data = confusion_matrix(Y_true, Y_pred_classes), fmt = "g", annot = True, cmap = "Blues")
plt.title('Confusion matrix')
plt.show()

Численные метрики

In [22]:
def pretty_table(data, cell_sep=' | ', header_separator=True) -> str:
    rows = len(data)
    cols = len(data[0])

    col_width = []
    for col in range(cols):
        columns = [str(data[row][col]) for row in range(rows)]
        col_width.append(len(max(columns, key=len)))

    separator = "-+-".join('-' * n for n in col_width)

    lines = []

    for i, row in enumerate(range(rows)):
        result = []
        for col in range(cols):
            item = str(data[row][col]).rjust(col_width[col])
            result.append(item)

        lines.append(cell_sep.join(result))

        if i == 0 and header_separator:
            lines.append(separator)

    return '\n'.join(lines)

In [23]:
data = [
    ["Метрика", "Значение"],
    {"Accuracy": round(accuracy_score(Y_true, Y_pred_classes), 3), 
     "Balanced accuracy": round(balanced_accuracy_score(Y_true, Y_pred_classes), 3), 
     "Average precision score": round(average_precision_score(Y_true, Y_pred_classes), 3),
     "Recall": round(recall_score(Y_true, Y_pred_classes, average='macro'), 3),
     "F1": round(f1_score(Y_true, Y_pred_classes, average='macro'), 3),}
]
rows = [
    data[0]
]
rows += [(k, v) for k, v in data[1].items()]
print(pretty_table(rows))

Accuracy

In [24]:
from matplotlib import pyplot

pyplot.plot(hist.history['accuracy'])
pyplot.title('Accuracy')
pyplot.show()

ROC-AUC

In [25]:
predictions = []
for i in Y_pred:
    predictions.append(np.amax(i)*100)

In [26]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve  
 
y = Y_true
pred = predictions
fpr, tpr, thresholds = roc_curve(y, pred, pos_label=1) 

from sklearn.metrics import auc  
AUC = auc(fpr, tpr) 

plt.plot(fpr, tpr)  
plt.title('ROC_curve' + '(AUC: ' + str(AUC) + ')' )
plt.ylabel('True Positive Rate')  
plt.xlabel('False Positive Rate')
pyplot.plot([0, 1], [0, 1], linestyle='--')
plt.show()  

Precision-Recall

In [27]:
from sklearn.metrics import precision_recall_curve
precision, recall, thresholds = precision_recall_curve(Y_true, predictions)

fig, ax = plt.subplots()
ax.plot(recall, precision, color='purple')

ax.set_title('Precision-Recall Curve')
ax.set_ylabel('Precision')
ax.set_xlabel('Recall')

plt.show()

**Предсказание**

In [28]:
class_names = ['Отфотошопленное', 'Оригинальное']

In [29]:
real_image_path = '/kaggle/input/casia-dataset/CASIA2/Au/Au_ani_00001.jpg'
image = prepare_image(real_image_path)
image = image.reshape(-1, 128, 128, 3)
y_pred = model.predict(image)
y_pred_class = np.argmax(y_pred, axis = 1)[0]
print(f'Class: {class_names[y_pred_class]} Confidence: {np.amax(y_pred) * 100:0.2f}')

In [ ]:
fake_image_path = '/kaggle/input/casia-dataset/CASIA2/Tp/Tp_D_NRN_S_N_ani10171_ani00001_12458.jpg'
image = prepare_image(fake_image_path)
image = image.reshape(-1, 128, 128, 3)
y_pred = model.predict(image)
y_pred_class = np.argmax(y_pred, axis = 1)[0]
print(f'Class: {class_names[y_pred_class]} Confidence: {np.amax(y_pred) * 100:0.2f}')

In [ ]:
fake_image = os.listdir('/kaggle/input/casia-dataset/CASIA2/Tp/')
correct = 0
total = 0
for file_name in fake_image:
    if file_name.endswith('jpg') or filename.endswith('png'):
        fake_image_path = os.path.join('/kaggle/input/casia-dataset/casia/CASIA2/Tp/', file_name)
        image = prepare_image(fake_image_path)
        image = image.reshape(-1, 128, 128, 3)
        y_pred = model.predict(image)
        y_pred_class = np.argmax(y_pred, axis = 1)[0]
        total += 1
        if y_pred_class == 0:
            correct += 1
#             print(f'Class: {class_names[y_pred_class]} Confidence: {np.amax(y_pred) * 100:0.2f}')

In [ ]:
print(f'Total: {total}, Correct: {correct}, Acc: {correct / total * 100.0}')

In [ ]:
real_image = os.listdir('/kaggle/input/casia-dataset/casia/CASIA2/Au/')
correct_r = 0
total_r = 0
for file_name in real_image:
    if file_name.endswith('jpg') or filename.endswith('png'):
        real_image_path = os.path.join('/kaggle/input/casia-dataset/CASIA2/Au/', file_name)
        image = prepare_image(real_image_path)
        image = image.reshape(-1, 128, 128, 3)
        y_pred = model.predict(image)
        y_pred_class = np.argmax(y_pred, axis = 1)[0]
        total_r += 1
        if y_pred_class == 1:
            correct_r += 1
#             print(f'Class: {class_names[y_pred_class]} Confidence: {np.amax(y_pred) * 100:0.2f}')

In [ ]:
correct += correct_r
total += total_r
print(f'Total: {total_r}, Correct: {correct_r}, Acc: {correct_r / total_r * 100.0}')
print(f'Total: {total}, Correct: {correct}, Acc: {correct / total * 100.0}')